# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 18, 2019**

In [1]:
using DataFrames # load package

## Манипуляции со столбцами в `DataFrame`

### Переименование столбцов

Давайте начнем с `DataFrame`` Bool`s, который имеет имена столбцов по умолчанию.

In [2]:
x = DataFrame(Bool, 3, 4)

,x1,x2,x3,x4
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


С помощью `rename` мы создаем новый` DataFrame`; здесь мы переименовываем столбец `: x1` в`: A`. (`rename` также принимает коллекции пар.)

In [3]:
rename(x, :x1 => :A)

,A,x2,x3,x4
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


С помощью `rename!` Мы выполняем преобразование на месте. 

На этот раз мы применили функцию к каждому имени столбца.

In [4]:
rename!(c -> Symbol(string(c)^2), x)

,x1x1,x2x2,x3x3,x4x4
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


Мы также можем изменить имя определенного столбца, не зная оригинала. 

Здесь мы меняем имя третьего столбца, создавая новый DataFrame.

In [5]:
rename(x, names(x)[3] => :third)

,x1x1,x2x2,third,x4x4
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


С помощью `names!` Мы можем изменить имена всех переменных.

In [6]:
names!(x, [:a, :b, :c, :d])

,a,b,c,d
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


Мы получаем ошибку, когда пытаемся указать повторяющиеся имена.

In [7]:
names!(x, fill(:a, 4))

ArgumentError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

если мы не передадим `makeunique = true`, что позволяет нам обрабатывать дубликаты в переданных именах.

In [8]:
names!(x, fill(:a, 4), makeunique=true)

,a,a_1,a_2,a_3
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,false,false,true
3,false,true,true,true


### Изменение порядка столбцов

Мы можем переупорядочить вектор names (x) по мере необходимости, создав новый DataFrame.

In [9]:
using Random
Random.seed!(1234)
x[shuffle(names(x))]

,a_1,a_3,a_2,a
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,false,true,false,false
3,true,true,true,false


Также можно использовать `permutecols!` Для достижения этой цели:

In [10]:
permutecols!(x, 4:-1:1); x

,a_3,a_2,a_1,a
,Bool,Bool,Bool,Bool
1,false,false,false,false
2,true,false,false,false
3,true,true,true,false


### Слияние / добавление столбцов

In [11]:
x = DataFrame([(i,j) for i in 1:3, j in 1:4])

,x1,x2,x3,x4
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


С помощью `hcat` мы можем объединить два` DataFrame`ы. Также поддерживается синтаксис [x y], но только в том случае, если у DataFrames уникальные имена столбцов.

In [12]:
hcat(x, x, makeunique=true)

,x1,x2,x3,x4,x1_1,x2_1,x3_1,x4_1
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 1)","(3, 2)","(3, 3)","(3, 4)"


Мы также можем использовать `hcat`, чтобы добавить новый столбец; для этого столбца будет использоваться имя по умолчанию `:x1`, поэтому в нашем случае требуется` makeunique = true`.

In [13]:
y = hcat(x, [1,2,3], makeunique=true)

,x1,x2,x3,x4,x1_1
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


Вы также можете добавить вектор с `hcat`.

In [14]:
hcat([1,2,3], x, makeunique=true)

,x1,x1_1,x2,x3,x4
,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


В качестве альтернативы вы можете добавить вектор со следующим синтаксисом. Это немного более многословно, но чище.

In [15]:
y = [x DataFrame(A=[1,2,3])]

,x1,x2,x3,x4,A
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


Здесь мы делаем то же самое, но добавляем столбец `:A` впереди.

In [16]:
y = [DataFrame(A=[1,2,3]) x]

,A,x1,x2,x3,x4
,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


Столбец также может быть добавлен в середине. Здесь используется метод грубой силы и создается новый объект DataFrame.

In [17]:
using BenchmarkTools
@btime [$x[1:2] DataFrame(A=[1,2,3]) $x[3:4]]

  7.325 μs (120 allocations: 9.36 KiB)


,x1,x2,A,x3,x4
,Tuple…,Tuple…,Int64,Tuple…,Tuple…
1,"(1, 1)","(1, 2)",1,"(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)",2,"(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)",3,"(3, 3)","(3, 4)"


Мы также можем сделать это с помощью специализированного метода «insertcols!». Давайте добавим `:newcol` к` DataFrame` `y`.

In [18]:
insertcols!(y, 2, newcol=[1,2,3])

,A,newcol,x1,x2,x3,x4
,Int64,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


Если вы хотите вставить одно и то же имя столбца несколько раз, необходимо сделать makeunique = true, как обычно.

In [19]:
insertcols!(y, 2, newcol=[1,2,3], makeunique=true)

,A,newcol_1,newcol,x1,x2,x3,x4
,Int64,Int64,Int64,Tuple…,Tuple…,Tuple…,Tuple…
1,1,1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)"
2,2,2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)"
3,3,3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)"


Мы можем видеть, насколько быстрее вставить столбец с помощью `insertcols!`, Чем с помощью `hcat`, используя` btime` (обратите внимание, что здесь мы используем обозначение `Pair` в качестве примера).

In [20]:
@btime insertcols!(copy($x), 3, :A => [1,2,3])

  893.476 ns (18 allocations: 1.41 KiB)


,x1,x2,A,x3,x4
,Tuple…,Tuple…,Int64,Tuple…,Tuple…
1,"(1, 1)","(1, 2)",1,"(1, 3)","(1, 4)"
2,"(2, 1)","(2, 2)",2,"(2, 3)","(2, 4)"
3,"(3, 1)","(3, 2)",3,"(3, 3)","(3, 4)"


Давайте использовать `insertcols!`, Чтобы добавить столбец на место,

In [21]:
insertcols!(x, ncol(x)+1, A=[1,2,3])

,x1,x2,x3,x4,A
,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


и поставить на место перед колонкой.

In [22]:
insertcols!(x, 1, B=[1,2,3])

,B,x1,x2,x3,x4,A
,Int64,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)",1
2,2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)",2
3,3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)",3


### Расщипление / удаление столбцов

Давайте создадим новый DataFrame `x` и покажем несколько способов создания DataFrames с подмножеством столбцов` x`.

In [23]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


Сначала мы можем сделать это по индексу:

In [24]:
x[[1,2,4,5]]

,x1,x2,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 4)","(3, 5)"


или по имени столбца:

In [25]:
x[[:x1, :x4]]

,x1,x4
,Tuple…,Tuple…
1,"(1, 1)","(1, 4)"
2,"(2, 1)","(2, 4)"
3,"(3, 1)","(3, 4)"


Мы также можем выбрать сохранение или исключение столбцов с помощью `Bool` (нам нужен вектор, длина которого равна числу столбцов в исходном объекте DataFrame).

In [26]:
x[[true, false, true, false, true]]

,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


Здесь мы создаем один столбец `DataFrame`,

In [27]:
x[[:x1]]

,x1
,Tuple…
1,"(1, 1)"
2,"(2, 1)"
3,"(3, 1)"


и здесь мы получаем доступ к вектору в столбце `:x1`.

In [28]:
x[:x1]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

In [29]:
x.x1 # the same

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

Мы могли бы получить тот же вектор по номеру столбца

In [30]:
x[1]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (3, 1)

и удалить все из `DataFrame` с помощью` empty! `.

In [31]:
empty!(y)

""
""


Здесь мы создаем копию `x` и удаляем 3-й столбец из копии с помощью` deletecols! `.

In [32]:
z = copy(x)
deletecols!(z, 3)

,x1,x2,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 4)","(3, 5)"


In [33]:
x

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


Обратите внимание, что вы также можете создать представление DataFrame, когда нам нужно подмножество его столбцов:

In [34]:
@btime x[[1,3,5]]

  932.889 ns (24 allocations: 1.94 KiB)


,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


In [35]:
@btime @view x[[1,3,5]]

  99.925 ns (4 allocations: 256 bytes)


,x1,x3,x5
,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 3)","(1, 5)"
2,"(2, 1)","(2, 3)","(2, 5)"
3,"(3, 1)","(3, 3)","(3, 5)"


(сейчас создание `view` идет медленно, но в следующих выпусках пакета DataFrames.jl оно станет значительно быстрее)

### Изменить столбец по имени

In [36]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


При использовании следующего синтаксиса существующий столбец изменяется без выполнения какого-либо копирования.

In [37]:
x[:x1] = x[:x2]
x

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


Мы также можем использовать следующий синтаксис для добавления нового столбца в конце DataFrame.

In [38]:
x[:A] = [1,2,3]
x

,x1,x2,x3,x4,x5,A
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Int64
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)",1
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)",2
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)",3


Новое имя столбца будет добавлено в наш DataFrame со следующим синтаксисом (7 равно `ncol (x) + 1`).

In [39]:
x[7] = 11:13
x

,x1,x2,x3,x4,x5,A,x7
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…,Int64,Int64
1,"(1, 2)","(1, 2)","(1, 3)","(1, 4)","(1, 5)",1,11
2,"(2, 2)","(2, 2)","(2, 3)","(2, 4)","(2, 5)",2,12
3,"(3, 2)","(3, 2)","(3, 3)","(3, 4)","(3, 5)",3,13


### Найти название столбца

In [40]:
x = DataFrame([(i,j) for i in 1:3, j in 1:5])

,x1,x2,x3,x4,x5
,Tuple…,Tuple…,Tuple…,Tuple…,Tuple…
1,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)"
2,"(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)"
3,"(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)"


Мы можем проверить, существует ли столбец с данным именем через

In [41]:
haskey(x, :x1)

true

и определить его индекс с помощью

In [42]:
findfirst(isequal(:x2), names(x))

2